In [52]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pandas as pd
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score

In [53]:
model = keras.models.load_model("../Saved Data/Model")
X_test = np.load('../Saved Data/Test Set/X_test.npy')
y_test = np.load('../Saved Data/Test Set/y_test.npy')

In [54]:
probabilities_vector = model.predict(X_test)
y_pred = np.argmax(probabilities_vector, axis=1)
np.savetxt('../Saved Data/Predicted Labels/python_model.txt', y_pred, fmt='%d')

28/28 [==============================] - 0s 10ms/step


In [55]:
accuracy_score = accuracy_score(y_test, y_pred)
precision_score = precision_score(y_test, y_pred, average='macro')
recall_score = recall_score(y_test, y_pred, average='macro')
f1_score = f1_score(y_test, y_pred, average='macro')
confusion_matrix = confusion_matrix(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

In [56]:
data = {"accuracy": accuracy_score, "precision": precision_score, "recall": recall_score, "f1": f1_score}
json_data = json.dumps(data, indent = 4)
json_file = open('../Saved Data/Temp Data/python_model.json', 'w')
json_file.write(json_data)
print(f"Classification Accuracy = {round(accuracy_score * 100, 2)}%\nPrecision = {round(precision_score * 100, 2)}%\nRecall = {round(recall_score * 100, 2)}%\nF1 Score = {round(f1_score * 100, 2)}%")

Classification Accuracy = 86.76%
Precision = 87.93%
Recall = 83.89%
F1 Score = 85.51%


In [57]:
print(classification_report)

              precision    recall  f1-score   support

           0       0.84      0.90      0.87        93
           1       0.92      0.79      0.85       107
           2       0.94      0.88      0.91        89
           3       0.84      0.85      0.85       139
           4       0.80      0.82      0.81       123
           5       0.94      0.69      0.79        45
           6       0.87      0.94      0.91       280

    accuracy                           0.87       876
   macro avg       0.88      0.84      0.86       876
weighted avg       0.87      0.87      0.87       876



In [58]:
print(confusion_matrix)

[[ 84   1   0   2   1   1   4]
 [  1  85   1   3   7   1   9]
 [  2   0  78   2   1   0   6]
 [  3   1   2 118   9   0   6]
 [  5   1   1   7 101   0   8]
 [  4   2   1   0   2  31   5]
 [  1   2   0   8   6   0 263]]


In [65]:
print(y_test.shape)
roc_auc_score(y_test, probabilities_vector, average=None, multi_class='ovr')

(876,)


array([0.98530603, 0.98021463, 0.99143383, 0.98122859, 0.96806271,
       0.96825779, 0.98649329])

In [60]:
print(probabilities_vector[0])

[1.2918953e-03 1.8762910e-03 7.5636856e-04 8.4774959e-04 3.4885199e-04
 4.3632707e-04 9.9444252e-01]
